In [1]:
import os
import pandas as pd
import numpy as np

from sys import stderr
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm  # For printing out progress bar
from nltk.corpus import stopwords # Import the stop word list
pd.set_option('display.max_colwidth', None)

In [2]:
stop_words = stopwords.words('english')

In [3]:
# Add some custom words to the list
stop_words.append('said')
stop_words.append('shall')
stop_words.append('ee')
stop_words.append('00')

In [4]:
# Read all folder names in the OCR (or a specified) directory
ocred_path = '/work/otb-lab/OCRed'
# ocred_path = '/Users/nitingupta/Desktop/OTB/OCRed'

years = [name for name in os.listdir(ocred_path) if not name.startswith('.')]
years.sort()
print(years)

['1868-69', '1869-1870', '1870-1871', '1871', '1871-1872', '1872-1873', '1873', '1873-1874', '1874', '1875-76', '1877-78', '1878', '1879', '1880', '1881-82', '1883', '1884', '1885', '1886-1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1958b', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968']


<br>
Read the final split csv...

In [5]:
# df = pd.read_csv('../Splitting/final-update/final-splits/final_splits.csv', index_col = 0)
df = pd.read_csv('../split/updated/results/final_splits.csv', index_col = 0)

In [6]:
df['year'] = df.index.str.split("_").str[0]

In [7]:
df

,law_type,state,sentence,length,start_page,end_page,act,section,path,year
id,,,,,,,,,,
1868-69_0000,Act,SOUTH CAROLINA,"AN ACT ACCEPTING THE BENEFITS OF “AN ACT DONATING PUBLIC LANDS TO THE SEVERAL STATES AND TERRITORIES WHICH MAY PROVIDE COLLEGES FOR THE BENEFIT OF AGRICULTURE AND THE MECHANIC ARTS,” APPROVED THE SECOND DAY OF JULY, IN THE YEAR OF OUR LORD ONE THOUSAND EIGHT HUNDRED AND SIXTYTWO.",285,71,71,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/071.jpg,1868-69
1868-69_0001,Act,SOUTH CAROLINA,"Whereas, by an Act of Congress, approved the twentythird day of July, in the year of our Lord one thousand eight hundred and sixtysix, entitled “ An Act to amend the fifth Section of an Act entitled ‘An Act donating public lands to the several States and Territories which may provide colleges for the benefit of agriculture and the mechanic arts,’” approved the second day of July, ig the year of our Lord one thousand eight hundred and sixtytwo, so as to extend the time within which the provisions of said Act shall be accepted and such colleges established, it was, among other things, by the Senate and House of Representatives of the United States of America, in Congress assembled, enacted that the time in which the several States may comply with the provisions of the said Act of July second, eighteen hundred and sixtytwo, entitled “An Act donating public lands to the several States and Territories which may provide colleges for the benefit of agriculture and the mechanic arts,” is extended so that the acceptance of the benefits of the said Act may be expressed within three years from the passage of the Act first above mentioned: Section 1.",1161,71,71,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/071.jpg,1868-69
1868-69_0002,Act,SOUTH CAROLINA,"Beit enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the State of South Carolina does hereby express its acceptance of the benefitsof the said Act of Congress, approved on the second day of July, in the year of our Lord one thousand eight hundred and sixtytwo, entitled “An Act donating public lands to the several States and Territories which may provide colleges for the benefit of agriculture and the mechanic arts,” and does hereby assent to the provisions in said Act contained, and to the conditions on which the grant of land and scrip by said Act authorized is made, and binds herself to the faithful performance of all the stipulations by her to be assumed in said Act contained; and it is further desired, that the State may be allowed to use the same for the establishment and support of a system of common free schools, if the State may so desire.",979,71,72,1,1,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/071.jpg,1868-69
1868-69_0003,Act,SOUTH CAROLINA,"Upon the passage of this Act, the Governor of the State is authorized to take such measures as he may deem necessary to secure the early realization of the benefits of the Act above mentioned.",197,72,72,1,2,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/072.jpg,1868-69
1868-69_0004,Act,SOUTH CAROLINA,"In the Senate House, the twentysecond day of July, in the year of our Lord one thousand eight hundred and sixtyeight.",122,72,72,1,1,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/072.jpg,1868-69
...,...,...,...,...,...,...,...,...,...,...
1968_6685,Act,SOUTH CAROLINA,Buy and sell real estate.,30,1000,1000,1374,8,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1968/images/1000.tiff,1968
1968_6686,Act,SOUTH CAROLINA,"Section 142563.1 of the

<br>

Make a dataframe that contains most frequent words (along with their counts) and the top tf-idf words (along with their scores) for each year.

In [8]:
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)

# Fit the vectorizer on all sentences
tfidf_vectorizer.fit(df['sentence'])

# Transform the sentences into TF-IDF vectors
tfidf_matrix = tfidf_vectorizer.transform(df['sentence'])

# Define the number of top terms to retrieve
top_n = 5

# Create an empty list to store dictionaries for each year's data
data_to_concat = []

In [9]:
# Group the DataFrame by 'year'
grouped = df.groupby('year')

# Set up the progress bar
progress_bar = tqdm(total=len(years), file=stderr)

# Iterate through each group (year)
for year, group in grouped:
    
    # Update the progress bar
    progress_bar.set_description(f"Processing year {year}")
        
    # Concatenate all sentences for the current year into one document
    year_text = ' '.join(group['sentence'])
    
    # Transform the concatenated text into a TF-IDF vector
    year_tfidf_vector = tfidf_vectorizer.transform([year_text])
    
    # Get the TF-IDF scores for the year
    year_tfidf_scores = year_tfidf_vector.toarray()[0]
    
    # Get the indices of the top N terms for the year
    ## row.argsort() calculates the indices that would sort the elements in ascending order
    ## [-top_n:] takes the last top_n elements from the sorted indices array
    ## [::-1] reverses the order of the selected indices to make the indices in descending order.
    top_indices = year_tfidf_scores.argsort()[-top_n:][::-1]
    
    # Get the top terms and their TF-IDF scores
    top_terms = [tfidf_vectorizer.get_feature_names_out()[i] for i in top_indices]
    top_scores = [year_tfidf_scores[i] for i in top_indices]
    
    # Round top_scores to 3 decimal places
    top_scores = list(np.around(np.array(top_scores),3))

    # Get the top n frequent words
    words_lower = pd.Series(year_text.lower().split())
    top_frequent = dict(words_lower[~words_lower.isin(stop_words)].value_counts()[:top_n])
    
    # Create a dictionary for the current year's data
    year_data = {'year': year, 'top frequent': top_frequent, 'top tf-idf': dict(zip(top_terms, top_scores))}
    
    # Append the dictionary to the list
    data_to_concat.append(year_data)
        
    # Update the progress bar
    progress_bar.update(1)
    
# Close the progress bar
progress_bar.set_description(f"Processed the list")
progress_bar.close()
    
# Concatenate all dictionaries into a single DataFrame
df_analysis = pd.DataFrame(data_to_concat)
df_analysis.set_index('year', inplace=True)

In [10]:
df_analysis

,top frequent,top tf-idf
year,,
1868-69,"{'county': 514, 'may': 471, 'state': 383, 'court': 268, 'sec.': 253}","{'state': 0.219, 'may': 0.192, 'court': 0.183, 'county': 0.181, 'src': 0.169}"
1869-1870,"{'state': 400, 'act': 372, 'may': 371, 'one': 249, 'county': 237}","{'state': 0.248, 'company': 0.224, 'act': 0.185, 'may': 0.182, '1870': 0.166}"
1870-1871,"{'may': 594, 'town': 504, 'state': 450, 'act': 394, 'county': 308}","{'town': 0.355, 'intendant': 0.21, 'council': 0.196, 'state': 0.187, 'may': 0.184}"
1871,"{'may': 2135, 'act': 1624, 'town': 1306, 'state': 1279, 'south': 1041}","{'town': 0.262, 'may': 0.204, '1872': 0.187, 'state': 0.179, 'act': 0.172}"
1871-1872,"{'may': 745, 'act': 577, 'town': 469, 'state': 463, 'south': 353}","{'town': 0.267, 'may': 0.203, 'state': 0.183, 'act': 0.171, 'dollars': 0.163}"
...,...,...
1964,"{'section': 2430, 'county': 1268, 'act': 1045, 'state': 896, 'may': 889}","{'section': 0.354, '1962': 0.202, 'county': 0.185, '1964': 0.174, 'code': 0.154}"
1965,"{'county': 6193, 'section': 4066, 'act': 2435, 'may': 2134, 'board': 2030}","{'county': 0.319, 'section': 0.224, 'laws1965': 0.224, 'district': 0.147, '1965': 0.144}"
1966,"{'section': 2897, 'county': 1868, 'act': 1305, 'state': 1127, 'board': 927}","{'section': 0.356, 'county': 0.232, '1962': 0.18, '1966': 0.173, 'act': 0.16}"


<br>